# IEEE-CIS fraud detection 

## 데이터 불러오기 

In [1]:
#불러오기 
import pandas as pd
from scipy import stats
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


In [7]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    #start_mem = df.memory_usage().sum() / 1024**2
    #print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    #end_mem = df.memory_usage().sum() / 1024**2
    #print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    #print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df


** featuring engineering ! **

1. train/test 합쳐서 전처리 및 feature engineering 진행 필요 

2. test data 합치기 전에 target 분리 

```python
features = train_transaction.iloc[:, 2:]
target = train_transaction.iloc[:, 1]
```


In [3]:
train_identity = pd.read_csv('/Users/krc/PycharmProjects/1_fraud_detection/data/train_identity.csv')
#전처리 한 버전으로 데려와야한다
train_transaction = pd.read_csv('/Users/krc/PycharmProjects/1_fraud_detection/data/0614_fraud_pp_ver3.csv')
test_transaction = pd.read_csv('/Users/krc/PycharmProjects/1_fraud_detection/data/test_transaction.csv')

train_transaction.head()

/var/folders/5m/j1nv4tbn34l6v5n5hjqzx0c40000gn/T/ipykernel_5836/1185328200.py:3: DtypeWarning: Columns (4,8,10,15,16,27,28,29,30,31,32,33,34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  train_transaction = pd.read_csv('/Users/krc/PycharmProjects/1_fraud_detection/data/0614_fraud_pp_ver3.csv')


,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321
0,2987000.0,0.0,86400.0,68.5,W,13926.0,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001.0,0.0,86401.0,29.0,W,2755.0,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002.0,0.0,86469.0,59.0,W,4663.0,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003.0,0.0,86499.0,50.0,W,18132.0,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004.0,0.0,86506.0,50.0,H,4497.0,514.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## modeling test

### Random forest

In [4]:
from sklearn.datasets import make_moons
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import mglearn

In [ ]:
#범주형 drop 
t_list = []
for i in range (len(tr_trans_pp.columns)):
    if tr_trans_pp.iloc[:,i].dtype != 'object':
        t_list.append(i)
tr_trans_pp_t = tr_trans_pp.iloc[:,t_list]      
tr_trans_pp_t.head()

In [ ]:
#nan drop 
tr_trans_pp_t = tr_trans_pp_t.drop(columns = ['dist1', 'dist2', 'D7','D8', 'D12', 'D14'] , axis =1 )
tr_trans_pp_t = tr_trans_pp_t.dropna()
tr_trans_pp_t.info()

In [ ]:
tr_trans_pp_col = tr_trans_pp_t.drop(columns = 'isFraud', axis =1 )
tr_trans_pp_lab = tr_trans_pp_t[['isFraud']]


In [ ]:
#데이터: 0613 pp ver 2 사용 (tr_trans_pp)

# 데이터셋 설정
X_train, X_test, y_train, y_test = train_test_split(tr_trans_pp_col, tr_trans_pp_lab, stratify=tr_trans_pp_lab,test_size = 0.2, random_state=42)


In [ ]:

# 모델 생성 및 학습
model = RandomForestClassifier(n_estimators=11, random_state=2).fit(X_train, y_train)
#number of estimator = tree의 개수, 모델 몇 개 쓸래?
#

In [ ]:
#평가


from sklearn.metrics import classification_report
pred = model.predict(X_test)
print(classification_report(y_test, pred))

In [ ]:
#feature importance
n_feature = X_train.shape[1]

index = np.arange(n_feature)



forest = RandomForestClassifier(n_estimators=100, n_jobs=-1)

forest.fit(X_train, y_train)

plt.barh(index, forest.feature_importances_, align='center')

plt.yticks(index, X_train.columns)

plt.ylim(-1, n_feature)

plt.xlabel('feature importance', size=15)

plt.ylabel('feature', size=15)

plt.show()

In [11]:
! pip install imblearn
from collections import Counter
def count_and_plot(y): 
    counter = Counter(y)
    for k,v in counter.items():
        print('Class=%d, n=%d (%.3f%%)' % (k, v, v / len(y) * 100))
    pyplot.bar(counter.keys(), counter.values())
    pyplot.show()

In [12]:
#undersampling

from imblearn.under_sampling import RandomUnderSampler
X_resampled, y_resampled = RandomUnderSampler(random_state=0).fit_resample(X_train, y_train)
count_and_plot(y_resampled)

NameError: name 'X_train' is not defined

### XG boost ( code copied)

In [4]:
!pip install xgboost

In [5]:
import warnings
import xgboost as xgb

print(f'xgboost=={xgb.__version__}')

from sklearn.metrics import confusion_matrix, classification_report

xgboost==1.6.1


In [6]:
def check_accuracy(y_answer, y_pred):
    accuracy = (y_answer == y_pred).sum() / y_pred.shape[0]
    return accuracy

In [10]:
#encoding 

from sklearn.preprocessing import LabelEncoder
from collections import defaultdict

class MultiColLabelEncoder:
    def __init__(self):
        self.encoder_dict = defaultdict(LabelEncoder)
        
    def fit_transform(self, X: pd.DataFrame, columns: list): # 컬럼명 리스트 기준으로 레이블인코딩
        if not isinstance(columns, list):
            columns = [columns]
        
        output = X.copy()
        output[columns] = X[columns].apply(lambda x: self.encoder_dict[x.name].fit_transform(x))
        
        return output
    
    def inverse_transform(self, X: pd.DataFrame, columns: list): # 인코딩 된 열 레이블 복구
        if not isinstance(columns, list):
            columns = [columns]
            
        if not all(key in self.encoder_dict for key in columns):
            raise KeyError(f'At least one of {columns} is not encoded before')
            
        output = X.copy()
        try:
            output[columns] = X[columns].apply(lambda x: self.encoder_dict[x.name].inverse_transform(x))
        except ValueError:
            print(f'Need assignment when do "fit_transform" function')
            raise
            
        return output

In [8]:
for i in range (len(train_transaction.iloc[0,:])):
    print(train_transaction.iloc[:, i].dtypes)
    print(train_transaction.iloc[:, i].name)

float64
TransactionID
float64
isFraud
float64
TransactionDT
float64
TransactionAmt
object
ProductCD
float64
card1
float64
card2
float64
card3
object
card4
float64
card5
object
card6
float64
addr1
float64
addr2
float64
dist1
float64
dist2
object
P_emaildomain
object
R_emaildomain
float64
C3
float64
C5
float64
C9
float64
C13
float64
D1
float64
D2
float64
D3
float64
D10
float64
D11
float64
D15
object
M1
object
M2
object
M3
object
M4
object
M5
object
M6
object
M7
object
M8
object
M9
float64
V56
float64
V95
float64
V96
float64
V97
float64
V99
float64
V101
float64
V102
float64
V103
float64
V105
float64
V106
float64
V126
float64
V127
float64
V128
float64
V129
float64
V130
float64
V131
float64
V132
float64
V133
float64
V134
float64
V135
float64
V136
float64
V137
float64
V279
float64
V280
float64
V282
float64
V283
float64
V285
float64
V290
float64
V291
float64
V292
float64
V293
float64
V294
float64
V295
float64
V296
float64
V298
float64
V299
float64
V306
float64
V307
float64
V308
float64
V309
f

In [8]:
#memory reduce
train_transaction = reduce_mem_usage(train_transaction)

In [ ]:
list_obj_trans = []
for i in range (len(train_transaction.iloc[0,:])) :
    if train_transaction.iloc[:, i].dtypes == 'object':
        list_obj_trans.append(train_transaction.iloc[:, i].name)
obj_to_int_trans = MultiColLabelEncoder().fit_transform(train_transaction, columns=list_obj_trans)
#obj_to_int_trans[list_obj_trans].head()
# obj_to_int_trans.head()
df_en = obj_to_int_trans[list_obj_trans]
df_en[['TransactionID']] = train_transaction[['TransactionID']]
df_en.head()

In [ ]:
#encoding 
df_left = tr_trans_pp.drop(columns = ['ProductCD', 'card4', 'card6', 'P_emaildomain','R_emaildomain','M1','M2','M3','M4','M5','M6','M7','M8','M9'], axis = 1)
tr_trans_pp_en = df_left.merge(df_en, how='inner', on='TransactionID')
tr_trans_pp_en.head()

In [ ]:
#drop 없이 모델 돌려보기
#데이터: 0613 pp ver 2, encoding 해서 사용 (tr_trans_pp)

# 데이터셋 설정
tr_trans_pp_col = tr_trans_pp_en.drop(columns = 'isFraud', axis =1 )
tr_trans_pp_lab = tr_trans_pp_en[['isFraud']]
X_train, X_test, y_train, y_test = train_test_split(tr_trans_pp_col, tr_trans_pp_lab, stratify=tr_trans_pp_lab,test_size = 0.2, random_state=42)


In [ ]:
xgb_clf = xgb.XGBClassifier(
    n_estimators = 10,
    max_depth = 4,
    booster = 'gbtree',
    eta = 0.3,
    gamma = 0,    
    silent = 0,
    objective = 'binary:logistic',
    nthread = 4,
    base_score = 0.5,    
)

xgb_clf.fit(X_train, y_train)
y_pred_xgb = xgb_clf.predict(X_test)
check_accuracy(y_pred_xgb, y_test.iloc[:,0])

### US - XGB (5배)

In [111]:
#undersampling

#Find Number of samples which are Fraud
no_frauds = len(tr_trans_p[tr_trans_p['isFraud'] == 1]) * 5 # 5배  
#Get indices of non fraud samples
non_fraud_indices = tr_trans_p[tr_trans_p.isFraud == 0].index
#Random sample non fraud indices
random_indices = np.random.choice(non_fraud_indices,no_frauds, replace=False)
#Find the indices of fraud samples
fraud_indices = tr_trans_p[tr_trans_p.isFraud == 1].index
#Concat fraud indices with sample non-fraud ones
under_sample_indices = np.concatenate([fraud_indices,random_indices])
#Get Balance Dataframe
under_sample = tr_trans_p.loc[under_sample_indices]

In [112]:
under_sample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 123978 entries, 203 to 278262
Data columns (total 43 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   TransactionID   123978 non-null  int64  
 1   isFraud         123978 non-null  int64  
 2   TransactionDT   123978 non-null  int64  
 3   TransactionAmt  123978 non-null  float64
 4   ProductCD       123978 non-null  object 
 5   card1           123978 non-null  int64  
 6   card2           122031 non-null  float64
 7   card3           123681 non-null  float64
 8   card4           123678 non-null  object 
 9   card5           123053 non-null  float64
 10  card6           123680 non-null  object 
 11  addr1           105808 non-null  float64
 12  addr2           105808 non-null  float64
 13  dist1           47114 non-null   float64
 14  dist2           9773 non-null    float64
 15  P_emaildomain   104560 non-null  object 
 16  R_emaildomain   33984 non-null   object 
 17  C2      

In [113]:
df = under_sample.copy()

In [114]:
list_obj_trans = []
for i in range (len(df.iloc[0,:])) :
    if df.iloc[:, i].dtypes == 'object':
        list_obj_trans.append(df.iloc[:, i].name)
obj_to_int_trans = MultiColLabelEncoder().fit_transform(df, columns=list_obj_trans)
#obj_to_int_trans[list_obj_trans].head()
# obj_to_int_trans.head()
df_en = obj_to_int_trans[list_obj_trans]
df_en[['TransactionID']] = df[['TransactionID']]
df_en.head()

/var/folders/5m/j1nv4tbn34l6v5n5hjqzx0c40000gn/T/ipykernel_3361/324271187.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_en[['TransactionID']] = df[['TransactionID']]


,ProductCD,card4,card6,P_emaildomain,R_emaildomain,M1,M2,M3,M4,M5,M6,M7,M8,M9,TransactionID
203,4,3,1,2,59,2,2,2,0,0,1,2,2,2,2987203
240,0,3,1,19,19,2,2,2,2,2,2,2,2,2,2987240
243,0,3,1,19,19,2,2,2,2,2,2,2,2,2,2987243
245,0,3,1,19,19,2,2,2,2,2,2,2,2,2,2987245
288,0,3,1,35,34,2,2,2,2,2,2,2,2,2,2987288


In [115]:
#encoding 
df_left = df.drop(columns = ['ProductCD', 'card4', 'card6', 'P_emaildomain','R_emaildomain','M1','M2','M3','M4','M5','M6','M7','M8','M9'], axis = 1)
tr_trans_pp_2 = df_left.merge(df_en, how='inner', on='TransactionID')
tr_trans_pp_2.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,dist1,dist2,C2,C3,C4,C5,C9,C13,D2,D3,D5,D7,D8,D11,D12,D14,V311,V313,V314,ProductCD,card4,card6,P_emaildomain,R_emaildomain,M1,M2,M3,M4,M5,M6,M7,M8,M9
0,2987203,1,89760,445.000,18268,583.0,150.0,226.0,251.0,87.0,NaN,NaN,2.0,0.0,0.0,0.0,2.0,4.0,57.0,52.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,4,3,1,2,59,2,2,2,0,0,1,2,2,2
1,2987240,1,90193,37.098,13413,103.0,185.0,137.0,NaN,NaN,NaN,NaN,1.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,45.041664,NaN,0.0,0.0,0.0,0.0,0.0,0,3,1,19,19,2,2,2,2,2,2,2,2,2
2,2987243,1,90246,37.098,13413,103.0,185.0,137.0,NaN,NaN,NaN,NaN,1.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,45.041664,NaN,0.0,0.0,0.0,0.0,0.0,0,3,1,19,19,2,2,2,2,2,2,2,2,2
3,2987245,1,90295,37.098,13413,103.0,185.0,137.0,NaN,NaN,NaN,NaN,1.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,45.041664,NaN,0.0,0.0,0.0,0.0,0.0,0,3,1,19,19,2,2,2,2,2,2,2,2,2
4,2987288,1,90986,155.521,16578,545.0,185.0,226.0,NaN,NaN,NaN,NaN,1.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0,3,1,35,34,2,2,2,2,2,2,2,2,2


In [116]:
#drop 없이 모델 돌려보기
#데이터: 0613 pp ver 2, encoding 해서 사용 (tr_trans_pp)

# 데이터셋 설정
tr_trans_pp_col = tr_trans_pp_2.drop(columns = 'isFraud', axis =1 )
tr_trans_pp_lab = tr_trans_pp_2[['isFraud']]
X_train, X_test, y_train, y_test = train_test_split(tr_trans_pp_col, tr_trans_pp_lab, stratify=tr_trans_pp_lab,test_size = 0.2, random_state=42)


In [117]:
xgb_clf = xgb.XGBClassifier(
    n_estimators = 10,
    max_depth = 4,
    booster = 'gbtree',
    eta = 0.3,
    gamma = 0,    
    silent = 0,
    objective = 'binary:logistic',
    nthread = 4,
    base_score = 0.5,    
)

xgb_clf.fit(X_train, y_train)
y_pred_xgb = xgb_clf.predict(X_test)
check_accuracy(y_pred_xgb, y_test.iloc[:,0])

[14:25:37] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-3.7/xgboost/src/learner.cc:627: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




0.8837715760606549

In [118]:

con_mat = confusion_matrix(y_pred_xgb, y_test.iloc[:,0])
print(">> confusion_matrix")
print(con_mat) #  tn, fp, fn, tp

>> confusion_matrix
[[20253  2472]
 [  410  1661]]


### US - XGB (2배)

In [ ]:
#PP import 
tr_trans_p = pd.read_csv('/Users/krc/PycharmProjects/1_fraud_detection/IEEE-CIS_Fraud_Detection/0613_fraud_pp_ver2.csv')

In [102]:
#undersampling

#Find Number of samples which are Fraud
no_frauds = len(tr_trans_p[tr_trans_p['isFraud'] == 1]) * 2 #열배! 
#Get indices of non fraud samples
non_fraud_indices = tr_trans_p[tr_trans_p.isFraud == 0].index
#Random sample non fraud indices
random_indices = np.random.choice(non_fraud_indices,no_frauds, replace=False)
#Find the indices of fraud samples
fraud_indices = tr_trans_p[tr_trans_p.isFraud == 1].index
#Concat fraud indices with sample non-fraud ones
under_sample_indices = np.concatenate([fraud_indices,random_indices])
#Get Balance Dataframe
under_sample = tr_trans_p.loc[under_sample_indices]

In [104]:
df = under_sample.copy()

In [105]:
list_obj_trans = []
for i in range (len(df.iloc[0,:])) :
    if df.iloc[:, i].dtypes == 'object':
        list_obj_trans.append(df.iloc[:, i].name)
obj_to_int_trans = MultiColLabelEncoder().fit_transform(df, columns=list_obj_trans)
#obj_to_int_trans[list_obj_trans].head()
# obj_to_int_trans.head()
df_en = obj_to_int_trans[list_obj_trans]
df_en[['TransactionID']] = df[['TransactionID']]
df_en.head()

/var/folders/5m/j1nv4tbn34l6v5n5hjqzx0c40000gn/T/ipykernel_3361/324271187.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_en[['TransactionID']] = df[['TransactionID']]


,ProductCD,card4,card6,P_emaildomain,R_emaildomain,M1,M2,M3,M4,M5,M6,M7,M8,M9,TransactionID
203,4,3,1,2,59,2,2,2,0,0,1,2,2,2,2987203
240,0,3,1,19,18,2,2,2,2,2,2,2,2,2,2987240
243,0,3,1,19,18,2,2,2,2,2,2,2,2,2,2987243
245,0,3,1,19,18,2,2,2,2,2,2,2,2,2,2987245
288,0,3,1,35,34,2,2,2,2,2,2,2,2,2,2987288


In [106]:
#encoding 
df_left = df.drop(columns = ['ProductCD', 'card4', 'card6', 'P_emaildomain','R_emaildomain','M1','M2','M3','M4','M5','M6','M7','M8','M9'], axis = 1)
tr_trans_pp_2 = df_left.merge(df_en, how='inner', on='TransactionID')
tr_trans_pp_2.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,dist1,dist2,C2,C3,C4,C5,C9,C13,D2,D3,D5,D7,D8,D11,D12,D14,V311,V313,V314,ProductCD,card4,card6,P_emaildomain,R_emaildomain,M1,M2,M3,M4,M5,M6,M7,M8,M9
0,2987203,1,89760,445.000,18268,583.0,150.0,226.0,251.0,87.0,NaN,NaN,2.0,0.0,0.0,0.0,2.0,4.0,57.0,52.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,4,3,1,2,59,2,2,2,0,0,1,2,2,2
1,2987240,1,90193,37.098,13413,103.0,185.0,137.0,NaN,NaN,NaN,NaN,1.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,45.041664,NaN,0.0,0.0,0.0,0.0,0.0,0,3,1,19,18,2,2,2,2,2,2,2,2,2
2,2987243,1,90246,37.098,13413,103.0,185.0,137.0,NaN,NaN,NaN,NaN,1.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,45.041664,NaN,0.0,0.0,0.0,0.0,0.0,0,3,1,19,18,2,2,2,2,2,2,2,2,2
3,2987245,1,90295,37.098,13413,103.0,185.0,137.0,NaN,NaN,NaN,NaN,1.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,45.041664,NaN,0.0,0.0,0.0,0.0,0.0,0,3,1,19,18,2,2,2,2,2,2,2,2,2
4,2987288,1,90986,155.521,16578,545.0,185.0,226.0,NaN,NaN,NaN,NaN,1.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0,3,1,35,34,2,2,2,2,2,2,2,2,2


In [107]:
#drop 없이 모델 돌려보기
#데이터: 0613 pp ver 2, encoding 해서 사용 (tr_trans_pp)

# 데이터셋 설정
tr_trans_pp_col = tr_trans_pp_2.drop(columns = 'isFraud', axis =1 )
tr_trans_pp_lab = tr_trans_pp_2[['isFraud']]
X_train, X_test, y_train, y_test = train_test_split(tr_trans_pp_col, tr_trans_pp_lab, stratify=tr_trans_pp_lab,test_size = 0.2, random_state=42)


In [110]:
xgb_clf = xgb.XGBClassifier(
    n_estimators = 10,
    max_depth = 4,
    booster = 'gbtree',
    eta = 0.3,
    gamma = 0,    
    silent = 0,
    objective = 'binary:logistic',
    nthread = 4,
    base_score = 0.5,    
)

xgb_clf.fit(X_train, y_train)
y_pred_xgb = xgb_clf.predict(X_test)
check_accuracy(y_pred_xgb, y_test.iloc[:,0])

con_mat = confusion_matrix(y_pred_xgb, y_test.iloc[:,0])
print(">> confusion_matrix")
print(con_mat) #  tn, fp, fn, tp

[14:23:33] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-3.7/xgboost/src/learner.cc:627: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


>> confusion_matrix
[[7673 1715]
 [ 592 2418]]


- XGB model 성능 향상을 위한 points
